In [2]:
#%pip install transformers[torch]
# %pip install torch
# %pip install datasets
#%pip install accelerate

In [3]:
import torch

DEVICE = "cpu"
# setup optimal acceleration DEVICE 
if torch.backends.mps.is_available():
    DEVICE = torch.device("mps")  # Use Metal Performance Shaders on macOS
elif torch.cuda.is_available():
    DEVICE = torch.device("cuda")  # to check if cuda is an option https://www.restack.io/p/gpu-computing-answer-is-my-gpu-cuda-enabled-cat-ai

print(f"DEVICE is : {DEVICE}")

DEVICE is : cuda


In [20]:
from transformers import AutoModelForMaskedLM, AutoTokenizer

def load_model_tokenizer(model_path_or_id:str, device:str):
    # Replace AutoModelForMaskedLM with the correct class for your task, e.g., AutoModelForSequenceClassification
    model = AutoModelForMaskedLM.from_pretrained(model_path_or_id)
    tokenizer = AutoTokenizer.from_pretrained(model_path_or_id)
    
    model.to(device)

    return model, tokenizer

In [5]:
model_checkpoint = 'distilbert/distilbert-base-uncased'

model, tokenizer = load_model_tokenizer(model_path_or_id=model_checkpoint, device=DEVICE)

In [6]:
#finetuning on IMDb
from datasets import load_dataset
imdb_dataset = load_dataset("imdb")
imdb_dataset["train"][0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [7]:
def tokenize_function(examples):
    result = tokenizer(examples["text"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result


# Use batched=True to activate fast multithreading!
tokenized_datasets = imdb_dataset.map(
    tokenize_function, batched=True, remove_columns=["text", "label"]
)
tokenized_datasets

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (519 > 512). Running this sequence through the model will result in indexing errors


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 50000
    })
})

In [8]:
chunk_size = tokenizer.model_max_length // 2
def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) if isinstance(examples[k][0], list) else examples[k] for k in examples.keys()}
    # Compute the total length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # Drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of chunk_size
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new "labels" column that is a copy of "input_ids"
    result["labels"] = result["input_ids"].copy()
    return result


In [9]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 30639
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 29946
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 61465
    })
})

In [10]:
from transformers import DataCollatorForLanguageModeling, TrainingArguments, Trainer

tokenizer.pad_token = "[PAD]" #tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [11]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="movie_model",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    push_to_hub=False,
    per_device_train_batch_size=32,  # lower this in case of memory problems
    fp16 = True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

#%time trainer.train()

In [12]:
def top_k_prediction(masked_text, model, tokenizer, k=10):
    model.to(DEVICE) 
    inputs = tokenizer(masked_text, return_tensors="pt").to(DEVICE)
    logits = model(**inputs).logits
    mask_token_index = torch.where(inputs["input_ids"]==tokenizer.mask_token_id)[1]
    mask_token_logits = logits[0, mask_token_index, :]
    return [tokenizer.decode(t) for t in torch.topk(mask_token_logits, k, dim=1).indices[0].tolist()]